Mount Google Drive , copy and extract data, checkout or pull to get the latest code from github.

In [62]:
import os, time

colab_data_dir = "/content/data/"

if not os.path.exists(colab_data_dir):
  !mkdir {colab_data_dir}
  print("Data folder created.")

if not os.path.exists("/gdrive"):
  print("Mounting google drive")
  from google.colab import drive
  drive.mount('/gdrive')
print("Google drive is mounted at /gdrive")
  
start_time = time.time()
if not os.path.exists(os.path.join(colab_data_dir, "300W-LP")):  
  print("Copying 300W-LP.zip...")
  !cp -n /gdrive/My\ Drive/colab_workspace/advertima/data/300W-LP.zip {colab_data_dir}
  print("Copied to {}\nExtracting...".format(colab_data_dir))
  !unzip -qq -n /content/data/300W-LP.zip -d  {colab_data_dir}  
  %rm {os.path.join(colab_data_dir, "300W-LP.zip")}
elapsed_time = time.time() - start_time
print("{} is ready ({:.1f} sec)".format(os.path.join(colab_data_dir, "300W-LP"), elapsed_time))
  
start_time = time.time()
if not os.path.exists(os.path.join(colab_data_dir, "AFLW2000-3D")):
  print("Copying AFLW2000-3D.zip...")
  %cp -n /gdrive/My\ Drive/colab_workspace/advertima/data/AFLW2000-3D.zip {colab_data_dir}
  print("Copied to {}\nExtracting...".format(colab_data_dir))
  !unzip -qq -n /content/data/AFLW2000-3D.zip -d {colab_data_dir}  
  %rm {os.path.join(colab_data_dir, "AFLW2000-3D.zip")}
elapsed_time = time.time() - start_time
print("{} is ready ({:.1f} sec)".format(os.path.join(colab_data_dir, "AFLW2000-3D"), elapsed_time))

%cd /content
if not os.path.exists("/content/deep-head-pose"):
  # Clone the entire repo.
  !git clone -l -s https://github.com/emredog/deep-head-pose.git deep-head-pose
  %cd deep-head-pose/code
else:
  %cd deep-head-pose/code
  !git pull

  


/content
/content/deep-head-pose/code
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 18 (delta 13), reused 11 (delta 9), pack-reused 0
Unpacking objects: 100% (18/18), done.
From https://github.com/emredog/deep-head-pose
   6122034..4868859  master     -> origin/master
Updating 6122034..4868859
Fast-forward
 Collab_test.ipynb     | 1744 +++++++++++++++++++++++++++++++++++++++++++++++--
 code/datasets.py      |  251 +++++--
 code/hopenet.py       |   31 +-
 code/train_hopenet.py |   12 +-
 code/utils.py         |  104 ++-
 5 files changed, 1995 insertions(+), 147 deletions(-)


In [0]:
import train_hopenet as trainer

In [67]:
!python train_hopenet.py --num_epochs 1 --batch_size 128 --lr 10e-5 --dataset Pose_300W_LP --data_dir /content/data --alpha 1 

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100% 102502400/102502400 [00:01<00:00, 102010721.58it/s]
Loading data.
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:209: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
Traceback (most recent call last):
  File "train_hopenet.py", line 106, in <module>
    pose_dataset = datasets.Pose_300W_LP(args.data_dir, args.filename_list, transformations)
  File "/content/deep-head-pose/code/datasets.py", line 126, in __init__
    filename_list = get_list_from_filenames(filename_path)
  File "/content/deep-head-pose/code/datasets.py", line 18, in get_list_from_filenames
    with open(file_path) as f:
FileNotFoundError: [Errno 2] No such file or directory: ''


In [68]:
!python train_hopenet.py --help


usage: train_hopenet.py [-h] [--gpu GPU_ID] [--num_epochs NUM_EPOCHS]
                        [--batch_size BATCH_SIZE] [--lr LR]
                        [--dataset DATASET] [--data_dir DATA_DIR]
                        [--filename_list FILENAME_LIST]
                        [--output_string OUTPUT_STRING] [--alpha ALPHA]
                        [--snapshot SNAPSHOT]

Head pose estimation using the Hopenet network.

optional arguments:
  -h, --help            show this help message and exit
  --gpu GPU_ID          GPU device id to use [0]
  --num_epochs NUM_EPOCHS
                        Maximum number of training epochs.
  --batch_size BATCH_SIZE
                        Batch size.
  --lr LR               Base learning rate.
  --dataset DATASET     Dataset type.
  --data_dir DATA_DIR   Directory path for data.
  --filename_list FILENAME_LIST
                        Path to text file containing relative paths for every
                        example.
  --output_string OUTPUT_STRING
  